<a href="https://colab.research.google.com/github/nlzzq/MDNA/blob/main/ComfyUIonColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#`mountUmount(`<font size="3px" color="#01c968">`Gdrive`</font>`)`



In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive!
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

#`Setup And Update ComfyUI`



In [ ]:
from pathlib import Path

OPTIONS = {}

DRIVE_PATH = ""  # @param {type:"string"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = '/content/ComfyUI'
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if DRIVE_PATH:

    WORKSPACE = DRIVE_PATH+"/ComfyUI"
    %cd {DRIVE_PATH}

![ ! -d WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

%cd {WORKSPACE}

-= Initial setup ComfyUI =-
Cloning into 'ComfyUI'...
remote: Enumerating objects: 33250, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 33250 (delta 126), reused 108 (delta 108), pack-reused 33028 (from 3)
Receiving objects: 100% (33250/33250), 78.60 MiB | 17.23 MiB/s, done.
Resolving deltas: 100% (22680/22680), done.
/content/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install dependencies =-
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121, https://download.pytorch.org/whl/cu118, https://download.pytorch.org/whl/cu117
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.2/21.2 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# `Models Download`

## Download MODELS

In [ ]:
%cd {WORKSPACE}
# Install Aria2
!apt-get -y install -qq aria2


from google.colab import userdata

CIVITAI_API_TOKEN = userdata.get('CIVITAI_API_TOKEN')
print("Loaded API key:", "✅" if CIVITAI_API_TOKEN else "❌ Not found")

# Install custom nodes

def install_custom_node(url):
  %cd /content/ComfyUI/custom_nodes
  !git clone {url}

def downloadModel(url, filename = None):
  if 'huggingface.co' in url:
    if filename is None:
      filename = url.split('/')[-1]
      filename = filename.removesuffix('?download=true')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    if filename:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}?token={CIVITAI_API_TOKEN} -o {filename}
    else:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}?token={CIVITAI_API_TOKEN}


# install custom nodes
install_custom_node('https://github.com/ltdrdata/ComfyUI-Manager.git')
install_custom_node('https://github.com/ltdrdata/ComfyUI-Impact-Pack')

# download models
%cd ./models/lora
downloadModel('https://civitai.com/api/download/models/2073605')
downloadModel('https://civitai.com/api/download/models/2176505')
downloadModel('https://civitai.com/api/download/models/2212384')
# Juppernaut X
#downloadModel('https://civitai.com/api/download/models/456194')
# Juggernaut v8
#downloadModel('https://civitai.com/api/download/models/288982')

# Huggingface example
#downloadModel('https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors?download=true')

%cd {WORKSPACE}

## OTHERS

In [ ]:
# --- 1. 创建目录结构 ---
!mkdir -p /content/ComfyUI/models/diffusion_models/
!mkdir -p /content/ComfyUI/models/text_encoders/
!mkdir -p /content/ComfyUI/models/vae/
!mkdir -p /content/ComfyUI/models/clip_vision/
!mkdir -p /content/ComfyUI/models/loras/SVI/

# --- 2. 下载 Wan 2.2 核心底模 (FP8 缩放版) ---
# 下载 High Noise 版本 (建议优先选择)
!wget -c "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_i2v_high_noise_14B_fp8_scaled.safetensors" -P /content/ComfyUI/models/diffusion_models/

# (可选) 下载 Low Noise 版本
# !wget -c "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_i2v_low_noise_14B_fp8_scaled.safetensors" -P /content/ComfyUI/models/diffusion_models/

# --- 3. 下载配套的 4 步快速生成 LoRA ---
!wget -c "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_i2v_lightx2v_4steps_lora_v1_high_noise.safetensors" -P /content/ComfyUI/models/loras/SVI/
!wget -c "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/loras/wan2.2_i2v_lightx2v_4steps_lora_v1_low_noise.safetensors" -P /content/ComfyUI/models/loras/SVI/

# --- 4. 下载通用必要组件 (VAE & 文本编码器) ---
# 根据工作流要求下载 UMT5 文本编码器
!wget -c "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors" -P /content/ComfyUI/models/text_encoders/

# 下载 Wan 2.1/2.2 通用 VAE
!wget -c "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors" -P /content/ComfyUI/models/vae/

# 下载 CLIP Vision (图生视频必用)
!wget -c "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/clip_vision/clip_vision_h.safetensors" -P /content/ComfyUI/models/clip_vision/

print("\n[完成] Wan 2.2 核心模型及配套组件已下载完毕！")

## LIST MODELS

In [ ]:
!ls -al ./models/checkpoints/

total 6775444
drwxr-xr-x  2 root root       4096 Aug 30 23:40 .
drwxr-xr-x 22 root root       4096 Aug 30 23:39 ..
-rw-r--r--  1 root root 6938040682 Aug 30 23:40 cyberrealisticPony_v127Alt.safetensors
-rw-r--r--  1 root root          0 Aug 30 23:39 put_checkpoints_here


# `START ComfyUI & Expose Server`

## Download Prerequisits

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

## CF Tunnel

In [ ]:
import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server